In [1]:
import numpy as np
import itertools
from environment.env import GridEnvDeform
from tqdm import trange

In [2]:
# maze size
N = 2

# thetas deformations (range(a,b),range(c,d))
l0 = 1
h0 = 10
l1 = 1
h1 = 10

maze = np.load(f"maze/maze_{N}.npy")

In [3]:
from stable_baselines3 import DQN
from environment.env import MDPGYMGridEnvDeform

In [9]:

from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecVideoRecorder

def make_env():
    N = 2

    # thetas deformations (range(a,b),range(c,d))
    l0 = 1
    h0 = 10
    l1 = 1
    h1 = 10
    
    maze = np.load(f"maze/maze_{N}.npy")
    env = MDPGYMGridEnvDeform(maze,l0,h0,l1,h1, render_mode="rgb_array")

    env = Monitor(env)  # record stats such as returns
    return env

env = DummyVecEnv([make_env])

env = VecVideoRecorder(env, video_folder="video", record_video_trigger=lambda x: x == 1,)

model= DQN.load("agents/pretrained/MDP/DQNsb3_z427ps8r.zip")



In [10]:
def eval_agent_mdp(agent,env,num_episodes,max_episode_steps,render):
    """Returns
        - episode_transition: list of list of tuples (s,a,r,s',done), t[i] is the ith episode
        - beliefs: list of beliefs at each time step 
    """
    
    # if render:
    #     env.set_rendering()

    for i in trange(num_episodes):

        s = env.reset()

        totalReward = 0.0
        done = False
        steps = 0
        episode_transitions = []

        while not done and steps < max_episode_steps:

            best_action, _ = agent.predict(s,deterministic=True)

            next_state, reward, done, info = env.step(best_action)

            totalReward += reward            

            if render:
                # print("State", s)
                # print("Action: ", best_action)
                # print("Reward:     " + str(totalReward) + "  ")
                # print("Next State: ", next_state)
                # print("\n")
            
                env.render()

            s = next_state
            steps += 1

eval_agent_mdp(model,env,10,100,False)

  0%|          | 0/10 [00:00<?, ?it/s]


ValueError: not enough values to unpack (expected 4, got 1)

In [5]:
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)

In [7]:
mean_reward, std_reward

(np.float64(-196.85), np.float64(4.95))